In [8]:
import cv2
import numpy as np


# Car[1,0]
# Human [0,1]

train_X = []
train_Y = []

test_X = []
test_Y = []

for i in range(1,5001):
    image = cv2.imread("Obje-Veri-Seti/car/car-"+str(i) + ".jpg") # Resmi okur.
    image = cv2.resize(image, (224,224)) # Resmi Yeniden Şekillendirir.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # Resmi Gri Tonlamaya Çevirir.
    Augment1 = cv2.flip(image, 1) # Resmi Aynalar.
    a,Augment2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY) # Resimi değiştirerek eğitim setini zenginleştirir.
    a,Augment3 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO) # Resimi değiştirerek eğitim setini zenginleştirir.
    train_X.append([image]) # Resmi Liste içine kaydeder
    train_X.append([Augment1]) # Resmi Liste içine kaydeder # Resmi Liste içine kaydeder
    train_X.append([Augment2]) # Resmi Liste içine kaydeder
    train_X.append([Augment3]) # Resmi Liste içine kaydeder
    
for i in range(1,5001):
    image = cv2.imread("Obje-Veri-Seti/human/human-"+str(i) + ".jpg")
    image = cv2.resize(image, (224,224))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    Augment1 = cv2.flip(image, 1) # Resmi Aynalar.
    a,Augment2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY) # Resimi değiştirerek eğitim setini zenginleştirir.
    a,Augment3 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO) # Resimi değiştirerek eğitim setini zenginleştirir.
    train_X.append([image]) # Resmi Liste içine kaydeder
    train_X.append([Augment1]) # Resmi Liste içine kaydeder # Resmi Liste içine kaydeder
    train_X.append([Augment2]) # Resmi Liste içine kaydeder
    train_X.append([Augment3]) # Resmi Liste içine kaydeder
    
    
for i in range(1,20001): # ilk 28700 resim Araba etiketi olan [1,0] ile etiketlenir
    train_Y.append([1,0])#Car
for i in range(1,20001):  # sonraki 28700 resim İnsan etiketi olan [0,1] ile etiketlenir
    train_Y.append([0,1])#Human
    



In [9]:
len(train_X) # Resimlerin sayısı

40000

In [10]:
train_X = np.array(train_X) # Resim listesini Numpy dizisine çevirmek.
train_X.reshape(-1,224,224,1) # Numpy dizisini Yapay Sinir Ağına vermek için yeniden çekillendirmek.
print(train_X.shape) # Şekillendirilen Train Data setinin şeklini yazdırmak.
train_Y = np.array(train_Y) # Etiket listesini numpy dizisine çevirmek
print(train_Y.shape)

X = np.array([i[0] for i in train_X]).reshape(-1,224,224,1)

(40000, 1, 224, 224)
(40000, 2)


In [11]:

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.optimizers import Adam

# Sequential modelin yaratılması
model = Sequential()

# 1.Convolutional Katman
model.add(Conv2D(filters=96, input_shape=(224,224,1), kernel_size=(11,11),\
 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Sonraki Katmana Geçmeden önce Batch Normalizasyonu 
model.add(BatchNormalization())

# 2. Convolutional Katman
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
#  Batch Normalizasyonu 
model.add(BatchNormalization())

# 3. Convolutional Katman
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalizasyonu 
model.add(BatchNormalization())

# 4. Convolutional Katman
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
#  Batch Normalizasyonu 
model.add(BatchNormalization())

# 5. Convolutional Katman
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
#  Batch Normalizasyonu 
model.add(BatchNormalization())

# Gelen verileri düz bir dizi haline getirmek.
model.add(Flatten())
# 1. Katman
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Aşırı Öğrenmeyi Engellemek İçin Dropout Eklemek.
model.add(Dropout(0.4))
#  Batch Normalizasyonu 
model.add(BatchNormalization())

# 2. Katman
model.add(Dense(4096))  # Nodelar
model.add(Activation('relu'))
# Dropout Eklemek
model.add(Dropout(0.4))
#  Batch Normalizasyonu 
model.add(BatchNormalization())

#3. Katman
model.add(Dense(1000)) # Nodelar
model.add(Activation('relu'))
# Droput eklenmesi.
model.add(Dropout(0.4))
#  Batch Normalizasyonu 
model.add(BatchNormalization())

# Çıkış Katmanı
model.add(Dense(2))
model.add(Activation('softmax')) # Çıkış softmax aktivasyon fonksiyonu.

model.summary() # Sinir Ağının genel Modeli

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 54, 54, 96)        11712     
_________________________________________________________________
activation_10 (Activation)   (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_11 (Activation)   (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 256)        

In [12]:
model.compile(optimizer="Adam",
             loss="categorical_crossentropy",
             metrics = ["accuracy"])
# Model eğitimi için categorical crossentropy kayıp fonksiyonu kullanılacak.
# Optimizer olarak Adam seçildi.
# Değer karşılaştırma metriği ise Doğruluğa göre yapılacak.

In [13]:
model.fit(X, train_Y,epochs= 5, batch_size = 128) 
# X data setini, train_Y etiketine göre eğit, 10 Kez üstünden geç, Aynı anda 64 veriyi değerlendir.

Epoch 1/5
40000/40000 [==============================] - 139s 3ms/step - loss: 0.3559 - accuracy: 0.8497
Epoch 2/5
40000/40000 [==============================] - 127s 3ms/step - loss: 0.1108 - accuracy: 0.9552
Epoch 3/5
40000/40000 [==============================] - 127s 3ms/step - loss: 0.0592 - accuracy: 0.9785
Epoch 4/5
40000/40000 [==============================] - 127s 3ms/step - loss: 0.0370 - accuracy: 0.9870
Epoch 5/5
40000/40000 [==============================] - 127s 3ms/step - loss: 0.0286 - accuracy: 0.9905


In [7]:
# testCar = cv2.imread("Obje-Veri-Seti/car/car-4110.jpg")
# testCar = cv2.resize(testCar, (224,224))
# testCar = cv2.cvtColor(testCar, cv2.COLOR_RGB2GRAY)

# testHuman = cv2.imread("Obje-Veri-Seti/human/human-4330.jpg")
# testHuman = cv2.resize(testHuman, (224,224))
# testHuman = cv2.cvtColor(testHuman, cv2.COLOR_RGB2GRAY)

# Test için veri hazırlanması


In [8]:
# testPen = np.array([testCar])
# model.predict(testPen.reshape(-1,224,224,1)) # hazırlanan verinin model üzerinde tahmin edilmesi

In [14]:
model.save('GTA(25.12.5).h5') # Eğitilen modelin kaydedilmesi